In [ ]:
with open("../data/nietzsche.txt", "r", encoding="utf8") as f:
    text = f.read().lower()

print(len(text))

In [ ]:
text[:500]

In [ ]:
import numpy as np
from tqdm import tqdm

maxlen = 60
step = 3
sentences = []
next_chars = []

for i in tqdm(range(0, len(text) - maxlen, step)):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
len(sentences)

In [ ]:
for i in range(5): print(repr(sentences[i])," : ", next_chars[i])

In [ ]:
chars = sorted(list(set(text)))
print(chars)

In [ ]:
char_indices = dict((char, chars.index(char)) for char in chars)
print(char_indices)

In [ ]:
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in tqdm(enumerate(sentences)):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

print("x", x.shape)
print("y", y.shape)

In [ ]:
import tensorflow as tf

model = tf.keras.Sequential([
    tf.keras.layers.GRU(16, input_shape=(maxlen, len(chars))),
    tf.keras.layers.Dense(units=len(chars), activation='softmax')
]) 

model.compile(loss='categorical_crossentropy', 
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.01))

model.summary()

In [ ]:
with tf.device("/CPU:0"):
    model.fit(x, 
            y,
            batch_size=128,
            epochs=1)

In [ ]:
import random

start_index = random.randint(0, len(text) - maxlen - 1)
base_text = text[start_index: start_index + maxlen]
base_text

In [ ]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [ ]:
import sys
temperatures = [0.2, 0.5, 1.0, 1.2]
gen_characters = 200

for temp in temperatures:
    print("Temp: ", temp)
    generated_text = base_text
    print(generated_text)
    for i in range(gen_characters):
        sampled = np.zeros((1, maxlen, len(chars)))    
        for t, char in enumerate(generated_text):
            sampled[0, t, char_indices[char]] = 1.          
        
        preds = model.predict(sampled, verbose=0)[0]      
        
        next_index = sample(preds, temp)
        next_char = chars[next_index]

        generated_text += next_char
        generated_text = generated_text[1:]

        sys.stdout.write(next_char)
        sys.stdout.flush()

    print()